In [ ]:
"""
Created on Wed Apr 26 13:05:27 2017

@author: zenodia.charpy
"""
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
import pandas as pd
#get the data
filname='fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
def getData(filname):
    # images are 48x48 = 2304 size vectors
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y) # scaling is already done here 
    return X, Y

X, Y = getData(filname)
print(X.shape)
print(Y.shape)
print(len(set(Y)))
num_class=len(set(Y))
#check if classes are balance
def balance_class(Y): 
    num_class=set(Y)
    count_class={}
    for i in range(len(num_class)):
        count_class[i]=sum([1 for y in Y if y==i])
    return count_class

balance=balance_class(Y)


#reshape X to fit keras with tensorflow backend
N,D=X.shape
X=X.reshape(N,48,48,1) # last dimension =1 is because it is black and white image, if colored, it will be 3

#split into training and testing set and rearrange the label y into 7 classes 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
y_train= (np.arange(num_class) == y_train[:,None]).astype(np.float32)
y_test=(np.arange(num_class) == y_test[:,None]).astype(np.float32)

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import *
from keras.layers import Dense
batch_size=30
epochs=20
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, border_mode='same',input_shape = (48, 48, 1), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (5, 5)))
#classifier.add(Dropout(0.25))
# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (5, 5)))
#classifier.add(Dropout(0.25))
# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(output_dim = 500, activation = 'relu')) # the output_dim is chosen by experience
classifier.add(Dense(output_dim = num_class, activation = 'sigmoid'))

# Compiling the CNN
sgd= SGD(lr=10e-5,momentum=0.99, decay=0.9999, nesterov=True)
classifier.compile(optimizer =sgd , loss = 'binary_crossentropy', metrics = ['accuracy'])
 
classifier.fit(X_train, y_train, 
           batch_size=batch_size, 
           epochs=epochs, 
           verbose=1, 
           validation_data=(X_test, y_test)) 
score = classifier.evaluate(X_test, y_test, verbose=0) 
print('Test loss:', score[0]) 
print('Test accuracy:', score[1]) 

In [ ]:
# save the model for later use
from keras.models import save_model, load_model
    
classifier.save('cnn_face_model.h5')  # creates a HDF5 file 
del classifier # deletes the existing model

# returns a compiled model
# identical to the previous one
#model =load_model('cnn_face_model.h5')


In [ ]:
# do necessary processing of the new image we get from googling 
﻿from keras.preprocessing import image
name_img='angry.jpg'
test_image=image.load_img(name_img, target_size =(48,48))
test_image=image.img_to_array(test_image)
test_image.shape

def rgb2gray(rgb): # turn the image into gray instead of having 3 colors

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray
test_image=rgb2gray(test_image)
test_image.shape

def new_img_convert(img):
    img=img.reshape((48,48,1))
    img/=255
    img=np.expand_dims(img, axis=0)
    return img

img=new_img_convert(test_image)
img.shape



In [ ]:
model =load_model('cnn_face_model.h5')
result=model.predict(img)
print (label_map)
print(result)